In [48]:
import cv2
import numpy as np
import shutil
import os
import mediapipe as mp
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [49]:
mp_holistic = mp.solutions.hands 
mp_drawing = mp.solutions.drawing_utils

In [50]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [51]:
def draw_styled_landmarks(image, results):
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

In [52]:
def extract_keypoints(results):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            matrx = []
            mark = hand_landmarks.landmark
            for res in mark:
                test = np.array([res.x, res.y, res.z])
                matrx.append(test)
            hand = np.array(matrx).flatten()
    else: hand = np.zeros(63)
        
    return np.concatenate([hand])

In [53]:
DATA_PATH = os.path.join('database')
actions=np.array([x for x in os.listdir("dataset/train")])

In [54]:

if os.path.exists(DATA_PATH) and os.path.isdir(DATA_PATH):
    shutil.rmtree(DATA_PATH)
    
for action in actions: 
    try:
        os.makedirs(os.path.join(DATA_PATH, action))
    except:
        print("the folders is exist or another Error")
        break

In [55]:
directory = "dataset/train"
with mp_holistic.Hands(static_image_mode=True,min_detection_confidence=0.8, min_tracking_confidence=0.5, ) as hands:
    for folder in os.listdir(directory):
        tmp = 0
        while tmp < 900:
            for i,img in enumerate(os.listdir(directory+'/'+folder)):
                train_dir = directory+'/'+folder+'/'+img
                sample_img = cv2.imread(train_dir)
                image, results = mediapipe_detection(sample_img, hands)
                keypoints = extract_keypoints(results)
                if results.multi_hand_landmarks:
                    npy_path = os.path.join(DATA_PATH, folder, str(tmp))
                    np.save(npy_path, keypoints)
                    tmp+=1
                if tmp >= 900:
                    break

In [56]:
label_map = {label:num for num, label in enumerate(actions)}
DATA_PATH = os.path.join('database')       
sequences, labels = [], []
for action in actions:
    window = []
    for num,sequence in enumerate(os.listdir(DATA_PATH +'/'+ action)):    
        res = np.load(os.path.join(DATA_PATH, action,sequence))
        window.append(res)
        if len(window) % 30 == 0 and num != 0:
            sequences.append(window)
            labels.append(label_map[action])
            window = []

In [57]:
print(np.array(labels).shape)
print(np.array(sequences).shape)

(90,)
(90, 30, 63)


In [58]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [59]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [60]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [61]:
model.fit(x_train, y_train, epochs=2000, validation_data=(x_test, y_test), callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 214ms/step - loss: 1.0987 - categorical_accuracy: 0.3289 - val_loss: 1.0966 - val_categorical_accuracy: 0.3571
Epoch 2/2000
3/3 [==============================] - 0s 66ms/step - loss: 1.0940 - categorical_accuracy: 0.4868 - val_loss: 1.0849 - val_categorical_accuracy: 0.6429
Epoch 3/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.0820 - categorical_accuracy: 0.5263 - val_loss: 1.0596 - val_categorical_accuracy: 0.5000
Epoch 4/2000
3/3 [==============================] - 0s 63ms/step - loss: 1.0529 - categorical_accuracy: 0.5263 - val_loss: 0.9932 - val_categorical_accuracy: 0.5714
Epoch 5/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.9669 - categorical_accuracy: 0.5658 - val_loss: 0.9337 - val_categorical_accuracy: 0.4286
Epoch 6/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.9891 - categorical_accuracy: 0.5395 - val_loss: 0.9552 - val_categorical_accuracy: 0.5000
Epoch 7/2

3/3 [==============================] - 0s 69ms/step - loss: 7.5527e-05 - categorical_accuracy: 1.0000 - val_loss: 0.3474 - val_categorical_accuracy: 0.9286
Epoch 51/2000
3/3 [==============================] - 0s 65ms/step - loss: 6.2643e-05 - categorical_accuracy: 1.0000 - val_loss: 0.3901 - val_categorical_accuracy: 0.9286
Epoch 52/2000
3/3 [==============================] - 0s 65ms/step - loss: 3.8232e-05 - categorical_accuracy: 1.0000 - val_loss: 0.4448 - val_categorical_accuracy: 0.9286
Epoch 53/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.9523e-05 - categorical_accuracy: 1.0000 - val_loss: 0.4952 - val_categorical_accuracy: 0.9286
Epoch 54/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.3892e-05 - categorical_accuracy: 1.0000 - val_loss: 0.5536 - val_categorical_accuracy: 0.9286
Epoch 55/2000
3/3 [==============================] - 0s 63ms/step - loss: 1.0066e-05 - categorical_accuracy: 1.0000 - val_loss: 0.6013 - val_categorical_accuracy: 0

3/3 [==============================] - 0s 63ms/step - loss: 2.7606e-07 - categorical_accuracy: 1.0000 - val_loss: 0.8135 - val_categorical_accuracy: 0.9286
Epoch 99/2000
3/3 [==============================] - 0s 62ms/step - loss: 2.6665e-07 - categorical_accuracy: 1.0000 - val_loss: 0.8147 - val_categorical_accuracy: 0.9286
Epoch 100/2000
3/3 [==============================] - 0s 62ms/step - loss: 2.6038e-07 - categorical_accuracy: 1.0000 - val_loss: 0.8158 - val_categorical_accuracy: 0.9286
Epoch 101/2000
3/3 [==============================] - 0s 64ms/step - loss: 2.5567e-07 - categorical_accuracy: 1.0000 - val_loss: 0.8169 - val_categorical_accuracy: 0.9286
Epoch 102/2000
3/3 [==============================] - 0s 63ms/step - loss: 2.4626e-07 - categorical_accuracy: 1.0000 - val_loss: 0.8179 - val_categorical_accuracy: 0.9286
Epoch 103/2000
3/3 [==============================] - 0s 63ms/step - loss: 2.3842e-07 - categorical_accuracy: 1.0000 - val_loss: 0.8190 - val_categorical_accurac

3/3 [==============================] - 0s 63ms/step - loss: 9.5681e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8550 - val_categorical_accuracy: 0.9286
Epoch 147/2000
3/3 [==============================] - 0s 65ms/step - loss: 9.4112e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8557 - val_categorical_accuracy: 0.9286
Epoch 148/2000
3/3 [==============================] - 0s 64ms/step - loss: 9.0975e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8564 - val_categorical_accuracy: 0.9286
Epoch 149/2000
3/3 [==============================] - 0s 63ms/step - loss: 8.9407e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8571 - val_categorical_accuracy: 0.9286
Epoch 150/2000
3/3 [==============================] - 0s 63ms/step - loss: 8.7838e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8578 - val_categorical_accuracy: 0.9286
Epoch 151/2000
3/3 [==============================] - 0s 63ms/step - loss: 8.6270e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8584 - val_categorical_accura

Epoch 194/2000
3/3 [==============================] - 0s 64ms/step - loss: 4.8625e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8789 - val_categorical_accuracy: 0.9286
Epoch 195/2000
3/3 [==============================] - 0s 63ms/step - loss: 4.7056e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8792 - val_categorical_accuracy: 0.9286
Epoch 196/2000
3/3 [==============================] - 0s 65ms/step - loss: 4.7056e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8795 - val_categorical_accuracy: 0.9286
Epoch 197/2000
3/3 [==============================] - 0s 62ms/step - loss: 4.5488e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8799 - val_categorical_accuracy: 0.9286
Epoch 198/2000
3/3 [==============================] - 0s 63ms/step - loss: 4.5488e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8804 - val_categorical_accuracy: 0.9286
Epoch 199/2000
3/3 [==============================] - 0s 64ms/step - loss: 4.3919e-08 - categorical_accuracy: 1.0000 - val_loss: 0.8808 - val_cat

...

Epoch 1770/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.5685e-09 - categorical_accuracy: 1.0000 - val_loss: 1.0110 - val_categorical_accuracy: 0.9286
Epoch 1771/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.5685e-09 - categorical_accuracy: 1.0000 - val_loss: 1.0111 - val_categorical_accuracy: 0.9286
Epoch 1772/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0112 - val_categorical_accuracy: 0.9286
Epoch 1773/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0112 - val_categorical_accuracy: 0.9286
Epoch 1774/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0112 - val_categorical_accuracy: 0.9286
Epoch 1775/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0112 - v

3/3 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0134 - val_categorical_accuracy: 0.9286
Epoch 1818/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0134 - val_categorical_accuracy: 0.9286
Epoch 1819/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0134 - val_categorical_accuracy: 0.9286
Epoch 1820/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0133 - val_categorical_accuracy: 0.9286
Epoch 1821/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0133 - val_categorical_accuracy: 0.9286
Epoch 1822/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.5685e-09 - categorical_accuracy: 1.0000 - val_loss: 1.0133 - val_categorical_a

Epoch 1865/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0162 - val_categorical_accuracy: 0.9286
Epoch 1866/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0163 - val_categorical_accuracy: 0.9286
Epoch 1867/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0163 - val_categorical_accuracy: 0.9286
Epoch 1868/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0163 - val_categorical_accuracy: 0.9286
Epoch 1869/2000
3/3 [==============================] - 0s 61ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0163 - val_categorical_accuracy: 0.9286
Epoch 1870/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0163 - v

3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0182 - val_categorical_accuracy: 0.9286
Epoch 1913/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.5685e-09 - categorical_accuracy: 1.0000 - val_loss: 1.0184 - val_categorical_accuracy: 0.9286
Epoch 1914/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0187 - val_categorical_accuracy: 0.9286
Epoch 1915/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0190 - val_categorical_accuracy: 0.9286
Epoch 1916/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0191 - val_categorical_accuracy: 0.9286
Epoch 1917/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0192 - val_categorical_a

Epoch 1960/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0225 - val_categorical_accuracy: 0.9286
Epoch 1961/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0226 - val_categorical_accuracy: 0.9286
Epoch 1962/2000
3/3 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0226 - val_categorical_accuracy: 0.9286
Epoch 1963/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0228 - val_categorical_accuracy: 0.9286
Epoch 1964/2000
3/3 [==============================] - 0s 60ms/step - loss: 1.5685e-09 - categorical_accuracy: 1.0000 - val_loss: 1.0229 - val_categorical_accuracy: 0.9286
Epoch 1965/2000
3/3 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 1.0231 - v

In [62]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 3)                 99        
                                                                 
Total params: 187,331
Trainable params: 187,331
Non-tr

In [63]:
model.save("action.h5")

In [64]:
sequence = []
sentence = []
threshold = 0.40
text=""

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        if np.all((keypoints == 0)) == False:
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 1: 
                    sentence = sentence[-1:]
            if len(sentence)> 0 :
                text = sentence[0]
        else:
            text = ""
            
        image = cv2.flip(image,1)
        cv2.putText(image, text, (3,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 265ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 18ms/step
rock
1/1 [==============================] - 0s 23ms/step
rock
1/1 [==============================] - 0s 23ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 18ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 17ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 20ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 19ms/step
rock
1/1 [==============================] - 0s 18ms/step
rock
1/1 [==============================] - 0s 18ms/step
rock
1/1 [=========================

paper
1/1 [==============================] - 0s 19ms/step
paper
1/1 [==============================] - 0s 19ms/step
paper
1/1 [==============================] - 0s 18ms/step
paper
1/1 [==============================] - 0s 19ms/step
paper
1/1 [==============================] - 0s 18ms/step
paper
1/1 [==============================] - 0s 19ms/step
paper
1/1 [==============================] - 0s 22ms/step
paper
1/1 [==============================] - 0s 18ms/step
scissors
1/1 [==============================] - 0s 18ms/step
scissors
1/1 [==============================] - 0s 20ms/step
scissors
1/1 [==============================] - 0s 19ms/step
scissors
1/1 [==============================] - 0s 18ms/step
scissors
1/1 [==============================] - 0s 19ms/step
scissors
1/1 [==============================] - 0s 21ms/step
scissors
1/1 [==============================] - 0s 20ms/step
scissors
1/1 [==============================] - 0s 20ms/step
scissors
